# Sliding Puzzle com Busca Gulosa

Como função heurística h(n) vamos usar Distância de Manhattan.

![1](https://github.com/alvaromfcunha-c210/aula6/blob/master/imagens/1.png?raw=1)

### Modelando o problema:

In [1]:
import numpy as np

class SlidingPuzzle:

  #Definindo a estrutura do meu sliding-puzzle
  #Ex: problema = SlidingPuzzle(3)
  #O parêmtro 3 significa que eu terei um puzzle 3x3
  def __init__(self, num_blocos):
    self.num_blocos = num_blocos

  #Busca a posição de um determinado elemento dentro de uma matriz estado.
  #Esse determinado elemento é o de valor 0
  #Se o elemento encontrado na matriz estado for 0, retornará a sua posição
  #OBS: Se encontrar o elemento, retorna a posição (i, j) correspondente; caso contrário, retorna (None, None).
  def _encontra_posicao(self, estado, elemento):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if elemento == estado[i, j]:
          return i, j
    return None, None

  #Compara duas matrizes atual e objetivo para verificar se são idênticas.
  #Percorre as posições das matrizes e verifica se o valor em cada posição é o mesmo.
  def verifica_estados(self, no_atual, objetivo):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if no_atual[i, j] != objetivo[i, j]:
          return False
    return True

  def expande_estados(self, no_atual):

    #Lista que armazenará os novos estados gerados ao mover a peça vazia nas diferentes direções (cima, baixo, esquerda e direita)
    # A cada novo estado gerado, ele é adicionado à lista novos_estados usando o método append.
    #Isso permite que a lista acumule todos os possíveis estados resultantes de um movimento válido da peça vazia.
    novos_estados = []

    #A função recebe uma matriz atual e calcula a posição da peça vazia (representada por 0) usando o método _encontra_posica
    linha, coluna = self._encontra_posicao(no_atual, 0)

    # Cima
    #Verifica se a peça vazia pode ser movida para cima (linha não é 0), cria um novo estado copiando a matriz atual
    #e trocando os valores da peça vazia e da peça acima dela.
    if linha != 0:
      nova_linha = linha - 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Baixo
    #Move a peça vazia para baixo, desde que a posição da linha não seja a última.
    if linha != self.num_blocos - 1:
      nova_linha = linha + 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Esquerda
    #Move a peça vazia para a esquerda, verificando se a coluna não é 0.
    if coluna != 0:
      nova_coluna = coluna - 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Direita
    # Move a peça vazia para a direita, desde que a posição da coluna não seja a última.
    if coluna != self.num_blocos - 1:
      nova_coluna = coluna + 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    return novos_estados

  #A distância de Manhattan é uma heurística comum usada em algoritmos de busca para estimar a distância entre dois estados.
  #Ela calcula a soma das distâncias horizontais e verticais entre cada bloco no estado atual e sua posição desejada no estado objetivo.
  def distancia_Manhattan(self, no_atual, objetivo):

    '''
    Calcula a distância de Manhattan entre o estado no_atual e o estado objetivo
    Args:
        - no_atual: estado no_atual do tabuleiro
        - objetivo: estado objetivo do problema
    Return:
        - Distância de Manhattan
    '''

    #soma das distâncias de Manhattan de todos os blocos em relação ao estado objetivo.
    distancia_total = 0

    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        bloco_atual = no_atual[i, j]
        linha, coluna = self._encontra_posicao(objetivo, bloco_atual)

        distancia = abs(linha - i) + abs(coluna - j)
        distancia_total += distancia

    return distancia_total

### Modelando a Busca Gulosa:

In [2]:
#Cria uma fila de prioridades, que é fundamental para a busca gulosa,
#onde os estados são explorados com base na prioridade calculada.
#OBS:A busca gulosa prioriza a expansão dos estados com as menores heurísticas de distância de Manhattan.
import heapq

class BuscaGulosa():
  def __init__(self, problema: SlidingPuzzle):
      self.problema = problema

  #verifica se um determinado estado já foi visitado, comparando-o com os estados
  #presentes na lista estados_visitados.
  def _verifica_visitado(self, estado, estados_visitados):
    for i in estados_visitados:
      if self.problema.verifica_estados(i, estado):
        return True
    return False


  def busca(self, inicio, fim):
    '''
      Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES
      Args:
          - inicio: estado inicial do problema
          - fim: estado objetivo
      Return:
          - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
    '''

    '''
      OBS.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
      a prioridade desse estado
    '''


    # fila: fila manipulada pela heapq.
    # fila de prioridades onde os estados com menores distâncias de Manhattan terão prioridade.
    fila = []

    #O identificador é usado para garantir a ordem dos estados na fila de prioridades quando a distância de Manhattan é igual.
    #Ela é uma espécie de "desempate" quando dois estados têm a mesma distância de Manhattan. Se escolhe então o estado com menor id
    #Ele começa em zero e é incrementado conforme novos estados são adicionados.
    id = 0

    # (distancia, id, estado): tupla inserida na fila sendo:
    #   - posição 0 a heurística;
    #   - posição 1 a um identificador;
    #       Caso nao tenha esse identificador é gerado um erro se for inserido estados com heurísticas iguais.
    #   - posição 2 o estado.

    #OBS: O parâmetro distância recebe o valor de 0, pois o estado inicial tem uma distância de Manhattan igual a 0
    #Logo ele é o próprio estado objetivo
    heapq.heappush(fila, (0, id, inicio))

    solucao_encontrada = False
    estados_visitados = []
    cont_estados = 0

    while not len(fila) == 0:

      #A função heappop é usada para remover e retornar o estado com a menor
      #distância de Manhattan da fila de prioridades. O [2] no final da expressão extrai o estado da tupla. Ou seja, o estado atual do meu puzzle
      no_atual = heapq.heappop(fila)[2]
      estados_visitados.append(no_atual)

      if self.problema.verifica_estados(no_atual, fim):
        solucao_encontrada = True
        break

      else:
        cont_estados += 1
        novos_estados = self.problema.expande_estados(no_atual)
        for novo_estado in novos_estados:
          if not self._verifica_visitado(novo_estado, estados_visitados):
            #A cada vez que um novo estado é inserido na fila, o id é incrementado
            id += 1
            #Calculando a distância de Manhattan do novo estado
            distancia = self.problema.distancia_Manhattan(novo_estado, fim)
            heapq.heappush(fila, (distancia, id, novo_estado))

    return solucao_encontrada, estados_visitados, cont_estados

### Resolvendo o problema:

In [3]:
problema = SlidingPuzzle(3)
bg = BuscaGulosa(problema)

inicio = np.array([[2, 8, 3],
                   [1, 6, 4],
                   [7, 0, 5]])

alvo = np.array([[1, 2, 3],
                 [8, 0, 4],
                 [7, 6, 5]])

solucao, visitados, passos = bg.busca(inicio, alvo)

from pprint import pprint

print(f'Solução encontrada? {solucao}')
print(f'Estados visitados:')
pprint(visitados)
print(f'Quantidade de passos: {passos}')

Solução encontrada? True
Estados visitados:
[array([[2, 8, 3],
       [1, 6, 4],
       [7, 0, 5]]),
 array([[2, 8, 3],
       [1, 0, 4],
       [7, 6, 5]]),
 array([[2, 0, 3],
       [1, 8, 4],
       [7, 6, 5]]),
 array([[0, 2, 3],
       [1, 8, 4],
       [7, 6, 5]]),
 array([[1, 2, 3],
       [0, 8, 4],
       [7, 6, 5]]),
 array([[1, 2, 3],
       [8, 0, 4],
       [7, 6, 5]])]
Quantidade de passos: 5
